In [1]:
import pandas as pd
import numpy as np

from  _products.__DEEPSOLAR_Resources import USRegions, US_Solar_Groupings, US_4Major, US_4Broken
from  _products.__DEEPSOLAR_Resources import model_varsD, state_abrev, conversions_d, hm_select, major4_reg, model_vars_HM
from  _products.__DEEPSOLAR_Resources import conversions_dHM, model_only, to_mm2, conversions_dHM
from  _products._DEEPSOLAR_ import remove_outliers3, standardize_select, report_var_stats, scale_selectNRM
from _products.utility_fnc import Alert_sounds, show_list, display_list, blocking_sound_player, display_dic
from _products.__PAPER_RESOURCES import bb_mod, pp_mod,HMcvr_dic, cvr_dicT, Util_vars,bb_mod2
big_beta_pth = r'C:\Users\gjone\Desktop\Convergence Stuff\Big_Beta_Sheet_DATA.xlsx'
regional_scld_pth = r'C:\Users\gjone\DeepLearningDeepSolar\_Data\_Regional_Scaled_Models'
scaled_pth = r'C:\Users\gjone\DeepLearningDeepSolar\_Data\_Scaled_Models'
source_file = r'C:\Users\gjone\DeepLearningDeepSolar\_Data\Mixed\MEGA\US_set_all_OMEGA_ANAD21_OLRC1_impTrgt.csv'
source_file = r'C:\Users\gjone\DeepLearningDeepSolar\_Data\Mixed\MEGA\US_set_all_OMEGA_1_24_21_Base.csv'
#source_file = r'C:\Users\gjone\DeepLearningDeepSolar\_Data\Mixed\MEGA\__CNRGNT_1_24_21_RFimpt.csv'
model_varsD = bb_mod2 +  ['High_Solar_Areas', 'Low_Solar_Areas', 'DS_HighSolar', 'Hot_Spots_hh', 'Hot_Spots_hown',
                             'Hot_Spots_AvgAr', ] + major4_reg


ds_df = pd.read_csv(source_file, low_memory=False)
ds_HM = pd.DataFrame(ds_df.copy())
#dfs = pd.DataFrame(ds_df.copy())
# ds_df.dropna(subset=["number_of_solar_system_per_household"], inplace=True)



def mm_unscaler(val, minval, maxval): 
    return val * (maxval - minval) + minval

def mm_scaler_val(og_val, minval, maxval):
    num =(og_val - minval)
    den = (maxval - minval)
    
    print('Num/denom: {}/{}'.format(num, den))
    #return (og_val - minval)/(maxval - minval) 
    return num/den 



def show_var_and_stats(var, minV, maxV):
    print('Variable: {}, Min: {}, Max: {}'.format(var, minV, maxV))
    return

min_max_dict = {}

    
    
def get_unscaled_val(val, variable, min_max_d=min_max_dict, ):
    show_var_and_stats(variable, min_max_dict[variable]['min'], min_max_dict[variable]['max'])
    return mm_unscaler(val, min_max_dict[variable]['min'], min_max_dict[variable]['max'])

def get_scaled_val(val, variable, min_max_d=min_max_dict, ):
    #print('Variable: {}, Min: {}, Max: {}'.format(variable, min_max_dict[variable]['min'], min_max_dict[variable]['max']))
    show_var_and_stats(variable, min_max_dict[variable]['min'], min_max_dict[variable]['max'])
    return mm_scaler_val(val, min_max_dict[variable]['min'], min_max_dict[variable]['max'])


def generate_min_max_dict(dfs, varsL, rdict=min_max_dict):
    for v in varsL:
        rdict[v]['min'] = dfs[v].min()
        rdict[v]['max'] = dfs[v].max()
        rdict[v]['mean'] = dfs[v].mean()
        rdict[v]['std'] = dfs[v].std()
    return rdict


def countmissing(df):
    missingsO = df.isna().sum()
    sz = df.shape[0]
    tot = 0
    rl = []
    print("Size: {}".format(sz))
    for cc in missingsO.index.tolist():
        cnt = missingsO[cc]
        if cnt > 0:
            print('{}: {}/{:.4}%'.format(cc, cnt, (cnt/sz)*100))
            rl.append(cc)
            tot += cnt
    print("Total missing: {} / {:.4}%".format(tot, (tot/sz)*100))
    return rl


columns_srt = sorted(list(set(ds_df.columns.tolist())))


def add_allval(df, val=0, name='New_Empty'):
    df[name] = np.full(len(df), val)
    return

def get_reg(v, rd):
    for k in rd:
        if v in rd[k]:
            return k
    print('\t\t\t\tUh Oh !!!!')
    return -1

regions_list = ['West', 'NorthEast', 'Mid West', 'South'] + ['Hot_Spots_hh','Hot_Spots_hown', 'Hot_Spots_AvgAr',] +['US',]
regions_list += ['High_Solar_Areas', 'Low_Solar_Areas',] + ['URBAN', 'locale_recode(rural)','locale_recode(suburban)',]

def display_store_regionals(df, regions, region_listd, model_vars, verbose=True):
    reg_di = {
        "Regional": ['West', 'NorthEast', 'Mid West', 'South'], 
        'Hot Spots': ['Hot_Spots_hh','Hot_Spots_hown', 'Hot_Spots_AvgAr',], 
        'US': ['US',],
        'Solar Groups': ['High_Solar_Areas', 'Low_Solar_Areas',],
        'Locale': ['URBAN', 'locale_recode(rural)','locale_recode(suburban)',],
    }
    
    vars_to_store=[]
    reg_re_d = {}
    
    for reg in regions:
        if verbose:
            print('\t\t\tThe Group: {}'.format(reg))
        gp = get_reg(reg, reg_di)
        if gp not in reg_re_d:
            reg_re_d[gp] = {}
        reg_re_d[gp][reg] = {}
        for v in model_vars: #+ ['URBAN', 'High_Solar_Areas', 'Low_Solar_Areas',] :
            if reg in ['US', ]:
                mu = df.loc[:, v].mean()
            else:
                mu = df.loc[df[reg] == 1, v].mean()
            if verbose:
                print("The average '{}',: {:.3f}".format(v, mu))
            reg_re_d[gp][reg][v] = [mu]
    return reg_re_d


def plot_regional_grouping_averages_from_dict(regdict, vars_to_plot, verbose=True):
    
    for group in regdict:
        if verbose:
            print("\t\t\t-----------   {}   ----------".format(group))
        title=group + ':\n' 
        plot_df = pd.DataFrame()
        for v in vars_to_plot:
            title2 = title + 'Variable: {}'.format(v.upper())
            
            for region in regdict[group]:
                if verbose:
                    print("\t\t----   {}   ----".format(region))
                    print("The average value of {}: for region {}: {}".format(v, region, regdict[group][region][v][0]))
                plot_df[region] = regdict[group][region][v]
            fgsz = 8
            ax = plot_df.plot.bar(title=title2, figsize=(fgsz,fgsz))
            mn = plot_df.values.min()
            mx = plot_df.values.max()
            if verbose:
                print('min max {}, {}'.format(mn, mx))
            sc = .01
            sc2 = .1
            mn = max(0, mn - mn*sc2)
            mx = mx + mx*sc
            if verbose:
                print('min max {}, {}\n'.format(mn, mx))
            
            ax.set_ylim([mn, mx])
    return
"""
def scale_select(df, to_scale, verbose=False):
    # Scale the data for the things we want to scale before we impute it
    for v in to_scale:
        if verbose:
            print('V: {}'.format(v))
        df[v] = (df[v].values - df[v].min()) / (df[v].max() - df[v].min())
    return df
"""
def regional_scale_select(df, regIDCOL, to_mm):
    regID = list(set(df[regIDCOL].tolist()))
    for v in to_mm:
        for r in regID:
            df.loc[df[regIDCOL] == r, v] = (df.loc[df[regIDCOL] == r, v] - df.loc[df[regIDCOL] == r, v].min())/(df.loc[df[regIDCOL] == r, v].max() - df.loc[df[regIDCOL] == r, v].min())


def show_list(l):
    for v in l:
        print("'{}',".format(v))
        
def calculate_perNZ(df, cn, cd, newcol, verbose=False):
    
    #df[newcol] = np.full(len(df), 0)
    rl = []
    cnt = 0
    weirdones = 0
    baddies = []
    for cnum, cdenom in zip(df[cn].tolist(), df[cd].tolist()):
        if cdenom == 0:
            if verbose:
                print('found a 0 at entry: {}'.format(cnt))
            rl.append(0)
            weirdones += 1
            baddies.append(cnt)
        else:
            rl.append(cnum/cdenom)
        cnt += 1
    df[newcol] = rl
    print("There were {} 0 entries for {}".format(weirdones, cd))
    return

def contradiction_seeker(df, binvar, valvar, verbose=False):
    cnt = 0
    baddies = []
    for a, b in zip(df[binvar].tolist(), df[valvar].tolist()):
        if a >= 1 and b <= 0:
            baddies.append(cnt)
            print("Found an Issue at entry: {}".format(cnt))
            print("{}: {}, {}: {}".format(binvar, a, valvar, b))
        cnt += 1
    print("There are {} contradicting entries".format(len(baddies)))
    return baddies    

def mark_majority_thresh(df, col, majthresh=.5):
    df[col + '_Maj'] = np.zeros(len(df))
    df.loc[df[col] > majthresh, col + '_Maj'] = 1
    return

def add_HOTSPOTS(df, var, new_var_name, percentile=.949, verbose=False):
    # get the value
    df[new_var_name] = np.zeros(len(df))
    threshold = df[var].quantile(percentile)
    if verbose:
        print('the {} percentile of {} is {}'.format(percentile, var, threshold))
    # generate the threshold
    df.loc[df[var] >= threshold, 'Hot_Spots_hh'] = 1
    return

show_list(columns_srt)

'%hh_size_1',
'%hh_size_2',
'%hh_size_3',
'%hh_size_4',
'Adoption',
'Adoption_25',
'Adoption_26hh',
'Adoption_26ho',
'Adoption_27hh',
'Adoption_27ho',
'Adoption_30hh',
'Adoption_30ho',
'Adoption_35hh',
'Adoption_35ho',
'Adoption_45hh',
'Adoption_45ho',
'Adoption_50',
'Adoption_50hh',
'Adoption_50ho',
'Anti_Occup',
'AvgSres',
'Cost_x_Consumption',
'DS_HighSolar',
'Gender_Ratio',
'Green_Travelers',
'Green_Travelers_log10',
'High_Solar_Areas',
'Hot_Spots_AvgAr',
'Hot_Spots_hh',
'Hot_Spots_hown',
'Inc_x_Consmpt_kwh',
'Income_x_College_Edu',
'Income_x_EnergyCost',
'Income_x_Suitable_m2',
'Low_Solar_Areas',
'Mid West',
'Mid_Agedwellings',
'NT3',
'NorthEast',
'PV_HuOwn',
'PV_per_100_HuOwn',
'Policy_Combo',
'Pro_Occup',
'SNRaPa',
'SNRaPcap',
'SNRpcap',
'SRaPa',
'SRaPcap',
'SRpcap',
'ST_pcap',
'Savings_potential',
'South',
'T3',
'Top_3_States',
'Tot_own_mw',
'URBAN',
'West',
'WestNT3',
'Yr_own_mwh',
'Yr_own_mwh_log10',
'Yrl_%_$_kwh',
'Yrl_%_inc',
'Yrl_savings_$',
'Yrl_savings_$_log10',
'active_

In [2]:
for f in model_varsD:
    print("'{}'".format(f))

'number_of_solar_system_per_household'
'pop_female'
'pop_male'
'female_pct'
'male_pct'
'Gender_Ratio'
'hispanic_pct'
'asian_pct'
'black_pct'
'white_pct'
'diversity'
'pop_med_age'
'population_density'
'urban_diversity'
'rural_diversity'
'suburban_diversity'
'popden_x_TotOK_cnt'
'popden_x_TotOK_RCnt'
'popden_x_TotOK_Rm2'
'educated_population_rate'
'education_bachelor_or_above_rate'
'education_bachelor_rate'
'education_college_rate'
'education_doctoral_rate'
'pop25_some_college_plus'
'household_type_family_rate'
'hh_gini_index'
'median_household_income'
'average_household_size'
'hu_own_pct'
'hu_med_val'
'daily_solar_radiation'
'cdd'
'cdd_std'
'climate_zone'
'household_count'
'hu_2000toafter_pct'
'locale_recode(rural)'
'locale_recode(suburban)'
'URBAN'
'solar_prod'
'total_own_Sbldg'
'total_own_devp'
'total_own_devpC'
'total_own_hh'
'Tot_own_mw'
'Yr_own_mwh'
'lowincome_tax_credit_bin'
'net_metering_bin'
'property_tax_bin'
'dlrs_kwh'
'Policy_Combo'
'total_own_elep'
'high_own_elep_hh'
'low_ow

In [3]:
tomany_missing = [
'pct_eli_hh',
'avg_months_tenancy',
'occ_rate',
'avg_ibi_pct',
'avg_cbi_usd_p_w',
'fmr_2br',
'total_units',
'active_subsidies',
'avg_pbi_usd_p_kwh',
'Adoption_27hh',
'Adoption_27ho',
'aqi_max',
'aqi_90th_percentile',
'aqi_median',
'political_ratio',
'voting_2012_dem_percentage',
'voting_2012_gop_percentage',   
]

In [4]:

for v in sorted(model_varsD):
    print('"{}",'.format(v))

"Cost_x_Consumption",
"DS_HighSolar",
"Gender_Ratio",
"High_Solar_Areas",
"Hot_Spots_AvgAr",
"Hot_Spots_hh",
"Hot_Spots_hown",
"Inc_x_Consmpt_kwh",
"Income_x_College_Edu",
"Income_x_EnergyCost",
"Income_x_Suitable_m2",
"Low_Solar_Areas",
"Mid West",
"NT3",
"NorthEast",
"Policy_Combo",
"Savings_potential",
"South",
"T3",
"Tot_own_mw",
"URBAN",
"West",
"WestNT3",
"Yr_own_mwh",
"Yrl_%_inc",
"Yrl_savings_$",
"al",
"ar",
"asian_pct",
"average_household_size",
"avg_monthly_consumption_kwh",
"az",
"black_pct",
"ca",
"cdd",
"cdd_std",
"climate_zone",
"co",
"ct",
"daily_solar_radiation",
"dc",
"de",
"diversity",
"dlrs_kwh",
"educated_population_rate",
"education_bachelor_or_above_rate",
"education_bachelor_rate",
"education_college_rate",
"education_doctoral_rate",
"female_pct",
"fl",
"ga",
"hh_gini_index",
"high_own_elep_hh",
"hispanic_pct",
"household_count",
"household_type_family_rate",
"hu_2000toafter_pct",
"hu_med_val",
"hu_own_pct",
"ia",
"id",
"il",
"in",
"income_x_consumption_energyCos

In [5]:
for v in sorted(model_varsD):
    print("'{}',".format(v))

'Cost_x_Consumption',
'DS_HighSolar',
'Gender_Ratio',
'High_Solar_Areas',
'Hot_Spots_AvgAr',
'Hot_Spots_hh',
'Hot_Spots_hown',
'Inc_x_Consmpt_kwh',
'Income_x_College_Edu',
'Income_x_EnergyCost',
'Income_x_Suitable_m2',
'Low_Solar_Areas',
'Mid West',
'NT3',
'NorthEast',
'Policy_Combo',
'Savings_potential',
'South',
'T3',
'Tot_own_mw',
'URBAN',
'West',
'WestNT3',
'Yr_own_mwh',
'Yrl_%_inc',
'Yrl_savings_$',
'al',
'ar',
'asian_pct',
'average_household_size',
'avg_monthly_consumption_kwh',
'az',
'black_pct',
'ca',
'cdd',
'cdd_std',
'climate_zone',
'co',
'ct',
'daily_solar_radiation',
'dc',
'de',
'diversity',
'dlrs_kwh',
'educated_population_rate',
'education_bachelor_or_above_rate',
'education_bachelor_rate',
'education_college_rate',
'education_doctoral_rate',
'female_pct',
'fl',
'ga',
'hh_gini_index',
'high_own_elep_hh',
'hispanic_pct',
'household_count',
'household_type_family_rate',
'hu_2000toafter_pct',
'hu_med_val',
'hu_own_pct',
'ia',
'id',
'il',
'in',
'income_x_consumption_energyCos

In [6]:
for v in sorted(conversions_d, key=lambda x: x[0]):
    print("'{}: {}',".format(v, conversions_d[v]))

'Adoption: Adoption',
'AvgSres: Avg_RPV_(m^2)',
'Gender_Ratio: Gender_Ratio',
'Inc_x_Consmpt_kwh: Inc_x_Consmpt_kwh',
'PV_HuOwn: PV_HuOwn',
'Policy_Combo: Policy_Combo',
'Yrl_savings_$: Estimated_Yearly_savings_$',
'age_median: Age_Median1',
'average_household_size: Average_Household_Size',
'avg_monthly_consumption_kwh: Avg_Monthly_Consumption_kwh',
'cdd: Cooling_Degree_Days',
'cdd_std: Cooling_Degree_Days_std',
'climate_zone: Climate_Zone',
'diversity: Racial_Diversity',
'daily_solar_radiation: Daily_Solar_Radiation',
'dlrs_kwh: Energy_Cost($/kWh)',
'education_college_rate: College_Edu',
'fips: fips',
'female_pct: Female_%',
'hh_gini_index: Income_Diversity',
'hdd_std: Heating_Degree_Days_std',
'hdd: Heating_Degree_Days',
'hu_2000toafter: Newer_Home',
'hu_own_pct: Home_owner_rt',
'hu_med_val: Household_Med_value',
'hu_monthly_owner_costs_lessthan_1000dlrs: Monthly_costs_l_1000',
'hu_monthly_owner_costs_greaterthan_1000dlrs: Monthly_costs_g_1000',
'locale_recode(rural): Rural',
'locale

In [7]:
#  things that are not some form of percentage and thus need to be normalized
to_mm2 = [
"Inc_x_Consmpt_kwh",
"Income_x_EnergyCost",
"Income_x_Suitable_m2",
"Income_x_College_Edu",
'Cost_x_Consumption',
"Savings_potential",
"Tot_own_mw",
"pop_female",
"pop_male",
"female_pct",
"male_pct",
'Gender_Ratio',
"Yr_own_mwh",
"Yrl_%_inc",
"Yrl_savings_$",
'median_household_income',
"average_household_size",
"avg_monthly_consumption_kwh",

'malePct_x_income', 
'malePop_x_income',    
'malePct_x_edu',
'malePop_x_edu',     
    
"cdd",
"cdd_std",

"daily_solar_radiation",

"dlrs_kwh",

"hh_gini_index",
"high_own_elep_hh",

"household_count",

"hu_med_val",

"income_x_consumption_energyCost",
"low_own_elep_hh",

"mod_own_elep_hh",
"ownership_x_TotOK_Rcnt",
"ownership_x_TotOK_Rm2",
"ownership_x_TotOK_cnt",
"pop25_some_college_plus",
"pop_med_age",
"popden_x_TotOK_RCnt",
"popden_x_TotOK_Rm2",
"popden_x_TotOK_cnt",
"population_density",
"rural_diversity",
"solar_prod",
"suburban_diversity",
"tn",
"total_own_Sbldg",
"total_own_devp",
"total_own_devpC",
"total_own_elep",
"total_own_hh",
'non_white',
"travel_time_average",
"urban_diversity",
"verylow_own_elep_hh",
]

dropNaList = [
"daily_solar_radiation",
"cdd_std", 
"rural", 
"URBAN",
'malePct_x_income', 
'malePop_x_income',   
"net_metering_bin",
"property_tax_bin",
"dlrs_kwh",
"Savings_potential",    

"avg_monthly_consumption_kwh",   
'Cost_x_Consumption',
    
"median_household_income",
"hu_med_val",
"average_household_size",

'malePct_x_edu',
'malePop_x_edu',   
'non_white',
"pop_female",
"pop_male",
"female_pct",
"male_pct",
"pop_med_age", 
'Gender_Ratio',
'education_college_rate',
"median_household_income",
]


In [8]:
model_varsD = sorted(list(set(model_varsD)))
for v in sorted(model_varsD):
    print('"{}":"{}",'.format(v, v))

"Cost_x_Consumption":"Cost_x_Consumption",
"DS_HighSolar":"DS_HighSolar",
"Gender_Ratio":"Gender_Ratio",
"High_Solar_Areas":"High_Solar_Areas",
"Hot_Spots_AvgAr":"Hot_Spots_AvgAr",
"Hot_Spots_hh":"Hot_Spots_hh",
"Hot_Spots_hown":"Hot_Spots_hown",
"Inc_x_Consmpt_kwh":"Inc_x_Consmpt_kwh",
"Income_x_College_Edu":"Income_x_College_Edu",
"Income_x_EnergyCost":"Income_x_EnergyCost",
"Income_x_Suitable_m2":"Income_x_Suitable_m2",
"Low_Solar_Areas":"Low_Solar_Areas",
"Mid West":"Mid West",
"NT3":"NT3",
"NorthEast":"NorthEast",
"Policy_Combo":"Policy_Combo",
"Savings_potential":"Savings_potential",
"South":"South",
"T3":"T3",
"Tot_own_mw":"Tot_own_mw",
"URBAN":"URBAN",
"West":"West",
"WestNT3":"WestNT3",
"Yr_own_mwh":"Yr_own_mwh",
"Yrl_%_inc":"Yrl_%_inc",
"Yrl_savings_$":"Yrl_savings_$",
"al":"al",
"ar":"ar",
"asian_pct":"asian_pct",
"average_household_size":"average_household_size",
"avg_monthly_consumption_kwh":"avg_monthly_consumption_kwh",
"az":"az",
"black_pct":"black_pct",
"ca":"ca",
"cdd

In [9]:
model_varsD = sorted(list(set(model_varsD)))
for v in sorted(model_varsD):
    print('"{}",'.format(v, v))

"Cost_x_Consumption",
"DS_HighSolar",
"Gender_Ratio",
"High_Solar_Areas",
"Hot_Spots_AvgAr",
"Hot_Spots_hh",
"Hot_Spots_hown",
"Inc_x_Consmpt_kwh",
"Income_x_College_Edu",
"Income_x_EnergyCost",
"Income_x_Suitable_m2",
"Low_Solar_Areas",
"Mid West",
"NT3",
"NorthEast",
"Policy_Combo",
"Savings_potential",
"South",
"T3",
"Tot_own_mw",
"URBAN",
"West",
"WestNT3",
"Yr_own_mwh",
"Yrl_%_inc",
"Yrl_savings_$",
"al",
"ar",
"asian_pct",
"average_household_size",
"avg_monthly_consumption_kwh",
"az",
"black_pct",
"ca",
"cdd",
"cdd_std",
"climate_zone",
"co",
"ct",
"daily_solar_radiation",
"dc",
"de",
"diversity",
"dlrs_kwh",
"educated_population_rate",
"education_bachelor_or_above_rate",
"education_bachelor_rate",
"education_college_rate",
"education_doctoral_rate",
"female_pct",
"fl",
"ga",
"hh_gini_index",
"high_own_elep_hh",
"hispanic_pct",
"household_count",
"household_type_family_rate",
"hu_2000toafter_pct",
"hu_med_val",
"hu_own_pct",
"ia",
"id",
"il",
"in",
"income_x_consumption_energyCos

In [10]:
mdf = report_var_stats(ds_df.copy(), sort_type='value', sort_list=['Missing'], ascending=False,
                               name=r'C:\Users\gjone\DeepLearningDeepSolar\___PresentationTools\_Stat_Tables\US_MODB1.xlsx')

skew index
 Index(['solar_system_count', 'total_panel_area', 'fips',
       'average_household_income', 'education_bachelor', 'education_college',
       'education_doctoral', 'education_high_school_graduate',
       'education_less_than_high_school', 'education_master',
       ...
       'median_household_income_log10', 'mod_own_devp_log10', 'hdd_log10',
       'mod_own_mwh_log10', 'cdd_log10', 'daily_solar_radiation_log10',
       'hu_med_val_log10', 'p16_employed_log10',
       'transportation_bicycle_rate_log10', 'travel_time_10_19_rate_log10'],
      dtype='object', length=572)
given df index
 RangeIndex(start=0, stop=72537, step=1)


In [11]:
for v in mdf.index.tolist():
    print("'{}',".format(v))

'pct_eli_hh',
'occ_rate',
'avg_months_tenancy',
'avg_ibi_pct',
'avg_cbi_usd_p_w',
'fmr_2br',
'active_subsidies',
'total_units',
'avg_pbi_usd_p_kwh',
'Adoption_27hh',
'Adoption_27ho',
'aqi_max',
'aqi_90th_percentile',
'aqi_median',
'political_ratio',
'voting_2012_gop_percentage',
'voting_2012_dem_percentage',
'cust_cnt',
'Savings_potential',
'daily_solar_radiation',
'cooling_design_temperature',
'heating_design_temperature',
'daily_solar_radiation_log10',
'hu_med_val',
'hu_med_val_log10',
'housing_unit_median_gross_rent',
'housing_unit_median_value',
'mortgage_with_rate',
'fam_med_income',
'med_inc_ebill_dlrs',
'Inc_x_Consmpt_kwh',
'Income_x_Suitable_m2',
'median_household_income',
'Income_x_EnergyCost',
'income_x_consumption_energyCost',
'Income_x_College_Edu',
'dlrs_kwh x median_household_income',
'hh_med_income',
'median_household_income_log10',
'Yrl_%_inc',
'hh_gini_index',
'average_household_income',
'avg_inc_ebill_dlrs',
'travel_time_average',
'number_of_solar_system_per_household

In [12]:
st_abrevs = state_abrev
# add the variables to the conversion dict
for v in model_varsD + st_abrevs:
    if v not in conversions_d:
        conversions_d[v] = v

# add the variables to the conversion dict
for v in hm_select + st_abrevs + Util_vars:
    if v not in conversions_dHM:
        conversions_dHM[v] = v

for v in model_only:
    min_max_dict[v] = {}
    min_max_dict[v]['min'] = ds_df[v].min()
    min_max_dict[v]['max'] = ds_df[v].max()
        
        
# set up the min max dictioanry for unscaleing later
min_max_dict = generate_min_max_dict(ds_df, varsL=model_only, rdict=min_max_dict)

for v in min_max_dict:
    print("\t\t\tv:{}".format(v))
    print('Min: {}'.format(min_max_dict[v]['min']))
    print('Mean: {}'.format(min_max_dict[v]['mean']))
    print('Max: {}'.format(min_max_dict[v]['max']))
    print('Std: {}'.format(min_max_dict[v]['std']))

			v:number_of_solar_system_per_household
Min: 0.0
Mean: 0.011469183891299504
Max: 4.5
Std: 0.042510025145511894
			v:PV_HuOwn
Min: 0.0
Mean: 0.028770900775861728
Max: 27.33333333333333
Std: 0.16067667367251298
			v:AvgSres
Min: 0.0
Mean: 17.842025952252445
Max: 74.98643328
Std: 13.012729986197485
			v:diversity
Min: 0.0
Mean: 0.3090801696401862
Max: 0.81244124
Std: 0.1994171705418032
			v:education_college_rate
Min: 0.0
Mean: 0.289880600293021
Max: 1.0
Std: 0.080064376569081
			v:female_pct
Min: 0.0
Mean: 0.50801594581017
Max: 1.0
Std: 0.04737945556210974
			v:age_median
Min: 11.3
Mean: 38.92695704173447
Max: 83.6
Std: 7.685293710478833
			v:population_density
Min: 0.0
Mean: 5354.667282919552
Max: 454706.9
Std: 11994.652915414403
			v:daily_solar_radiation
Min: 3.3
Mean: 4.254363227691616
Max: 5.68
Std: 0.5543577329751439
			v:cdd
Min: 42.03694452
Mean: 1723.896704966233
Max: 5184.807896
Std: 993.5793047704998
			v:locale_recode(rural)
Min: 0.0
Mean: 0.302907481698995
Max: 1.0
Std: 0.

In [13]:
# quick adjust

ds_df['malePct_x_income'] = ds_df['male_pct'] * ds_df['median_household_income']
ds_df['malePop_x_income'] = ds_df["pop_male"] * ds_df['median_household_income']

ds_df['malePct_x_edu'] = ds_df['male_pct'] * ds_df['education_college_rate']
ds_df['malePop_x_edu'] = ds_df["pop_male"] * ds_df['education_college_rate']



In [14]:
ds_df['non_white'] = 1 - ds_df['white_pct']

print(ds_df['non_white'].mean())
print(ds_df['diversity'].mean())

print(ds_df['non_white'][0])
print(ds_df['diversity'][0])
print(1 - ds_df['white_pct'][0])



0.2728585974848387
0.3090801696401862
0.02387106583548715
0.04696006
0.02387106583548715


In [15]:
#                                        Base Model set
impt = False
new_set = pd.DataFrame()
hm_set = pd.DataFrame()

# rmv outliers
outlier_var2 = 'PV_HuOwn'

print('\t\t\t\t\t -----------   Performing outlier removal   -----------\n\n')
ds_df = remove_outliers3(ds_df, threshold=2, vars_to_check=[outlier_var2], verbose=True)
ds_df.dropna(subset=["number_of_solar_system_per_household"] , inplace=True)


# Scale desired variables on the most data possible
ds_df = scale_selectNRM(ds_df, to_mm2, verbose=False) 
#ds_df = standardize_select(ds_df, to_mm2, verbose=False)
# drop missing targets
#ds_df.dropna(subset=["number_of_solar_system_per_household"], inplace=True)


# now convert the names by generating new data frame
for v in model_varsD + st_abrevs + ['malePct_x_income', 'malePop_x_income','malePct_x_edu', 'malePop_x_edu', 'non_white']:
    if v not in conversions_d:
        conversions_d[v] = v
    new_set[conversions_d[v]] = ds_df[v]
    

impStr = 'NOIMP'
# impute if desired
if impt:
    newSet_Missing = countmissing(new_set)
    new_set.dropna(subset=newSetMissing, inplace=True)
    impStr = ''
new_set.to_excel(regional_scld_pth + '\{}'.format('PaperSet_3_3_OLR_NRML_FF_{}.xlsx'.format(impStr)))

blocking_sound_player(Alert_sounds[2])
dest = r'C:\Users\gjone\Documents\ArcGIS\Projects\Convergent_Solar_Maps\_DataSources'
geoids = []

for f in ds_HM['fips']:
    f = str(f)
    if len(f)%2 == 0:
        f = '0' + f
    geoids.append(f)
ds_HM['GEOID'] = geoids
    
for v in model_vars_HM + st_abrevs + Util_vars:
    hm_set[conversions_dHM[v]] = ds_HM[v]


print("\t\t\t    Heat Map set")
display(hm_set.describe())
hm_set.to_excel(dest + '\PaperSetTop5_US_HM.xlsx', index=False)
blocking_sound_player(Alert_sounds[1])

					 -----------   Performing outlier removal   -----------


V: PV_HuOwn
bool vals: [False False False ... False False False]
Out count: 469

checking PV_HuOwn
before removal shape 72537
Now it is 72068
a 0.6465665798144395 % loss
---------------------------------

11N: 72537, cleaned: 72068, 0.647% data loss
PV_HuOwn lost 0.6465665798144395 %
			    Heat Map set


,RPV/HH,RPV/HOw,AvgSize,fips,Diverse%,College%,Female%,Med_Age,SolarRad,cdd,...,West,Mid West,South,NorthEast,High_Solar_Areas,Low_Solar_Areas,DS_HighSolar,Hot_Spots_AvgAr,Hot_Spots_hh,Hot_Spots_hown
count,71766.000000,72537.000000,72537.000000,7.253700e+04,71940.000000,71930.000000,71940.000000,71907.000000,66735.000000,72265.000000,...,72537.000000,72537.000000,72537.000000,72537.000000,72537.000000,72537.000000,72537.000000,72537.0,72537.000000,72537.0
mean,0.011469,0.028771,17.842026,2.794758e+10,0.309080,0.289881,0.508016,38.926957,4.254363,1723.896705,...,0.215035,0.235645,0.337828,0.211492,0.395770,0.524243,0.286378,0.0,0.115003,0.0
std,0.042510,0.160677,13.012730,1.579265e+10,0.199417,0.080064,0.047379,7.685294,0.554358,993.579305,...,0.410850,0.424404,0.472973,0.408369,0.489019,0.499415,0.452071,0.0,0.319028,0.0
min,0.000000,0.000000,0.000000,1.001020e+09,0.000000,0.000000,0.000000,11.300000,3.300000,42.036945,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
25%,0.000000,0.000000,0.000000,1.212709e+10,0.129020,0.238669,0.488632,33.700000,3.790000,1029.864875,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
50%,0.001678,0.004430,20.734971,2.716307e+10,0.286196,0.292208,0.509410,38.900000,4.110000,1400.298500,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0
75%,0.007792,0.022989,26.827879,4.104303e+10,0.479223,0.342525,0.531074,43.700000,4.610000,2200.422844,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.0,0.000000,0.0
max,4.500000,27.333333,74.986433,5.604595e+10,0.812441,1.000000,1.000000,83.600000,5.680000,5184.807896,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,0.0


In [16]:
for v in model_varsD:
    print(v)

Cost_x_Consumption
DS_HighSolar
Gender_Ratio
High_Solar_Areas
Hot_Spots_AvgAr
Hot_Spots_hh
Hot_Spots_hown
Inc_x_Consmpt_kwh
Income_x_College_Edu
Income_x_EnergyCost
Income_x_Suitable_m2
Low_Solar_Areas
Mid West
NT3
NorthEast
Policy_Combo
Savings_potential
South
T3
Tot_own_mw
URBAN
West
WestNT3
Yr_own_mwh
Yrl_%_inc
Yrl_savings_$
al
ar
asian_pct
average_household_size
avg_monthly_consumption_kwh
az
black_pct
ca
cdd
cdd_std
climate_zone
co
ct
daily_solar_radiation
dc
de
diversity
dlrs_kwh
educated_population_rate
education_bachelor_or_above_rate
education_bachelor_rate
education_college_rate
education_doctoral_rate
female_pct
fl
ga
hh_gini_index
high_own_elep_hh
hispanic_pct
household_count
household_type_family_rate
hu_2000toafter_pct
hu_med_val
hu_own_pct
ia
id
il
in
income_x_consumption_energyCost
ks
ky
la
locale_recode(rural)
locale_recode(suburban)
low_own_elep_hh
lowincome_tax_credit_bin
ma
male_pct
md
me
median_household_income
mi
mn
mo
mod_own_elep_hh
ms
mt
nc
nd
ne
net_metering_b

In [17]:
ds_df['me']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
72532    0.0
72533    0.0
72534    0.0
72535    0.0
72536    0.0
Name: me, Length: 71297, dtype: float64

In [18]:
for v in hm_select:
    print(v)
hm_set[conversions_dHM['Inc_x_Consmpt_kwh']]

Adoption
number_of_solar_system_per_household
PV_HuOwn
AvgSres
state
fips
GEOID
state_fips
state_abbr
locale_recode(rural)
locale_recode(suburban)
hu_2000toafter
median_household_income
net_metering_bin
property_tax_bin
Policy_Combo
avg_monthly_consumption_kwh
travel_time_30_39_rate
High_Solar_Areas
Low_Solar_Areas
Hot_Spots_hh
Hot_Spots_hown
Hot_Spots_AvgAr
URBAN
locale_recode(rural)
locale_recode(suburban)
Inc_x_Consmpt_kwh


0        8.882127e+07
1        6.375198e+07
2        8.114040e+07
3        4.547597e+07
4        4.344477e+07
             ...     
72532    4.977516e+07
72533    4.310686e+07
72534    5.090280e+07
72535    5.339702e+07
72536    3.984467e+07
Name: IncxCons, Length: 72537, dtype: float64

In [19]:
HM_des = hm_set.describe()
idx = HM_des.index.tolist()
print(idx)
sz = len(hm_set)
print("\t\t\t\tmissing:")
for c in hm_select:
    c = conversions_dHM[c]
    print("\nlooking for {}".format(c))
    if c not in HM_des.columns:
        continue
    missings = sz - HM_des.loc['count', c]
    print("{}: {}".format(c, missings))
    

['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
				missing:

looking for Adoption

looking for RPV/HH
RPV/HH: 771.0

looking for RPV/HOw
RPV/HOw: 0.0

looking for AvgSize
AvgSize: 0.0

looking for State

looking for fips
fips: 0.0

looking for GEOID

looking for state_fips

looking for state_abbr

looking for Rural
Rural: 0.0

looking for Suburban
Suburban: 0.0

looking for HuVge2k
HuVge2k: 0.0

looking for MedHhInc
MedHhInc: 957.0

looking for NetMeter
NetMeter: 0.0

looking for PrpTax
PrpTax: 0.0

looking for Pol_Com
Pol_Com: 0.0

looking for AvgMthCn
AvgMthCn: 272.0

looking for CmmtH_Rt
CmmtH_Rt: 705.0

looking for High_Solar_Areas
High_Solar_Areas: 0.0

looking for Low_Solar_Areas
Low_Solar_Areas: 0.0

looking for Hot_Spots_hh
Hot_Spots_hh: 0.0

looking for Hot_Spots_hown
Hot_Spots_hown: 0.0

looking for Hot_Spots_AvgAr
Hot_Spots_AvgAr: 0.0

looking for URBAN

looking for Rural
Rural: 0.0

looking for Suburban
Suburban: 0.0

looking for IncxCons
IncxCons: 957.0


In [20]:
newSet_Missing = countmissing(new_set)
print()
print(newSet_Missing)

Size: 71297
Inc_x_Consmpt_kwh: 166/0.2328%
Income_x_College_Edu: 166/0.2328%
Income_x_EnergyCost: 166/0.2328%
Income_x_Suitable_m2: 166/0.2328%
Savings_potential: 5547/7.78%
Yrl_%_inc: 107/0.1501%
Average_Household_Size: 32/0.04488%
Climate_Zone: 3/0.004208%
Daily_Solar_Radiation: 5547/7.78%
education_bachelor_rate: 3/0.004208%
College_Edu: 3/0.004208%
education_doctoral_rate: 3/0.004208%
Income_Diversity: 69/0.09678%
Household_Med_value: 2249/3.154%
income_x_consumption_energyCost: 166/0.2328%
Median_Household_income: 166/0.2328%
Age_Median: 30/0.04208%
Popden_x_TotOK_RCnt: 24/0.03366%
Popden_x_TotOK_Rm2: 24/0.03366%
Popden_x_TotOK_cnt: 24/0.03366%
Population_Density: 24/0.03366%
travel_time_10_19_rate: 17/0.02384%
Average_Commute: 122/0.1711%
travel_time_less_than_10_rate: 17/0.02384%
malePct_x_income: 166/0.2328%
malePop_x_income: 166/0.2328%
malePct_x_edu: 3/0.004208%
malePop_x_edu: 3/0.004208%
Total missing: 15179 / 21.29%

['Inc_x_Consmpt_kwh', 'Income_x_College_Edu', 'Income_x_E

In [21]:
new_set_des = new_set.describe()

row_names = new_set_des.index.tolist()
std_dict = {}
for v in new_set_des.columns.tolist():
    print('\n\t\t\t"{}",'.format(v))
    #for r in row_names:
    #    print('{}:{}'.format(r, new_set_des.loc[r, v]))
    print('{}:{}'.format('std', new_set_des.loc['std', v]))
    std_dict[v] = new_set_des.loc['std', v]
    
rt_vars = ["Racial_Diversity","College_Edu_%","Female_%","Newer_Home%","Home_owner_rt", "Occupied_owner_rate", 
          "Commutes__40min_%",]


			"Cost_x_Consumption",
std:0.10293554398757675

			"DS_HighSolar",
std:0.4509174686668372

			"Gender_Ratio",
std:0.010158842538620979

			"High_Solar_Areas",
std:0.4885803182456941

			"Hot_Spots_AvgAr",
std:0.0

			"Hot_Spots_hh",
std:0.31258278348585666

			"Hot_Spots_hown",
std:0.0

			"Inc_x_Consmpt_kwh",
std:0.08384881110968118

			"Income_x_College_Edu",
std:0.08688618047169515

			"Income_x_EnergyCost",
std:0.08262870797605916

			"Income_x_Suitable_m2",
std:0.02901679592438024

			"Low_Solar_Areas",
std:0.49919256229284037

			"Mid West",
std:0.4254845377269566

			"NT3",
std:0.34547583165928597

			"NorthEast",
std:0.40878735398363064

			"Policy_Combo",
std:1.3547286410808943

			"Savings_potential",
std:0.04024376930338194

			"South",
std:0.47306361801840124

			"T3",
std:0.34547583165928597

			"Tot_own_mw",
std:0.03718014291563567

			"URBAN",
std:0.4840270129521952

			"West",
std:0.4090340979867918

			"WestNT3",
std:0.26159053894619044

			"Yr_own_mwh",
std:0.03490

In [22]:
old_vd = {}
un_cnvrt = {v: k for k, v in zip( list(conversions_d.keys()),list(conversions_d.values()))}
for v in std_dict:
    vu = un_cnvrt[v]
    if vu not in min_max_dict:
        continue
    scl_v = std_dict[v]
    variable_to_scale = vu
    if v not in rt_vars:
        oldv = get_unscaled_val(scl_v, variable=variable_to_scale, min_max_d=min_max_dict)
        print('unscaled: {}'.format(oldv))
        
    else:
        oldv = dfs[vu].std()
        print('V: {}'.format(vu))
        print('original rate std: {}'.format(oldv))
    print("\n")
    old_vd[v] = oldv

weight_df = pd.read_excel(big_beta_pth, sheet_name='Big Beta', index_col='v')

group_dict = {}
for group in weight_df.columns.tolist():
    print("\n\t\t\t-----  {}  -----".format(group))
    group_dict[group] = {}
    weight_df.sort_values(by=group, inplace=True)
    for v in weight_df.index.tolist():
        group_dict[group][v] = {}
        beta = weight_df.loc[v, group]
        std = old_vd[v]
        group_dict[group][v]['Beta'] = beta
        group_dict[group][v]['std'] = beta
        print('for a {} increase in {} --> {} change in per hh rate'.format(std, v, beta))
    
    

Variable: Inc_x_Consmpt_kwh, Min: 2288483.2159785, Max: 326828819.978848
unscaled: 29500804.61068066


Variable: Yrl_savings_$, Min: 0.0, Max: 19404300.3399246
unscaled: 783492.5863470726


Variable: average_household_size, Min: 1.02, Max: 11.0
unscaled: 1.522504042537744


Variable: avg_monthly_consumption_kwh, Min: 203.8461538, Max: 2535.364312
unscaled: 472.88917335117424


Variable: cdd, Min: 42.03694452, Max: 5184.807896
unscaled: 1035.9646695977538


Variable: daily_solar_radiation, Min: 3.3, Max: 5.68
unscaled: 3.8587252797953884




NameError: name 'dfs' is not defined

In [ ]:
tdf = pd.DataFrame({
    "a":[1, 2, 3, 4],
    'b':[5, 6, 7, 8]
})

print(tdf)

np.random.shuffle(tdf.values)

print(tdf)

In [ ]:
tdf2 = pd.DataFrame({
    "a":[1, 2, 3, 4],
    'b':[5, 6, 7, 8]
})

print(tdf2.mean())
print(tdf2.std())
print(tdf2.describe())


In [ ]:
tdf3 = pd.DataFrame({
    "a":[1, 2, 3, 4, np.nan, np.nan, np.nan],
    'b':[5, 6, 7, 8, np.nan, np.nan, np.nan]
})

print(tdf3.mean())
print(tdf3.std())
print(tdf3.describe())